# Run all 

In [1]:
import sys, os
import pandas as pd
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), ".."))) # lägg till training i path
from src.data_loader import load_local_data, unload_local_data
from src.scrapers.abstract_scraper import AbstractScraper
from src.scrapers.afry_scraper import AfryScraper
from src.scrapers.aliant_scraper import AliantScraper
from src.scrapers.asociety_scraper import ASocietyScraper
from src.scrapers.cinode_scraper import CinodeScraper
from src.scrapers.combitech_scraper import CombitechScraper
from src.scrapers.emagine_scraper import EmagineScraper
from src.scrapers.ework_scraper import EworkScraper
from src.scrapers.itcnetwork_scraper import ITCNetworkScraper
from src.scrapers.nikita_scraper import NikitaScraper
from src.scrapers.profinder_scraper import ProfinderScraper
from src.scrapers.regent_scraper import RegentScraper
from src.scrapers.senterprise_scraper import SenterpriseScraper
from src.scrapers.sigma_scraper import SigmaScraper
from src.scrapers.tingent_scraper import TingentScraper
from src.scrapers.upgraded_scraper import UpgradedScraper


scrapers = [AfryScraper(), AliantScraper(), ASocietyScraper(), CinodeScraper(), CombitechScraper(), EmagineScraper(), EworkScraper(), ITCNetworkScraper(), NikitaScraper(), ProfinderScraper(), RegentScraper(), SenterpriseScraper(), SigmaScraper(), TingentScraper(), UpgradedScraper()]
scrapers = [ProfinderScraper()] #Cindoe
#scrapers = []

nr_ads_pre = len(load_local_data())

for s in scrapers:
    response = s.request_status()
    scraped_payload_dict = s.scrape_jobs_payloads_dict(response)

    old_payload_data = load_local_data(file_name='jobs_payloads.csv')
    old_payload_dict = dict(zip(old_payload_data['id'], old_payload_data['raw_payload']))
    old_bronze_data = load_local_data()
    
    new_payload_dict = s.return_new_payloads(new_dict=scraped_payload_dict, old_dict=old_payload_dict)
    new_bronze_data = s.parse_bronze_data(new_payload_dict)

    updated_payload_dict = s.concat_dicts(new_payload_dict, old_payload_dict)
    updated_payload_data = pd.DataFrame(list(updated_payload_dict.items()), columns=['id','raw_payload'])
    updated_bronze_data = s.concat_new_rows(new_bronze_data, old_bronze_data)

    #unload_local_data(updated_payload_data, file_name='jobs_payloads.csv')
    #unload_local_data(updated_bronze_data)

nr_ads_post = len(pd.read_csv('../data/bronze/jobs.csv'))
print('Total number of new added jobs:', nr_ads_post-nr_ads_pre)
new_bronze_data


Profinder > Response: 200
Profinder > Nmr of scraped adds: 24
Profinder > Parsing bronze data: 24
Total number of new added jobs: 0


,id,site,site_id,job_title,area,due_date,work_location,work_type,link,ingestion_ts,is_new
0,Profinder-https://www.profinder.se/post/engine...,Profinder,https://www.profinder.se/post/engineering-proj...,Engineering Project (Senior Level) – Uppsala,None,None,None,None,https://www.profinder.se/post/engineering-proj...,2025-09-23 21:09:42,True
1,Profinder-https://www.profinder.se/post/ssdlc-...,Profinder,https://www.profinder.se/post/ssdlc-architect-...,SSDLC Architect – Embedded Software Security,None,None,None,None,https://www.profinder.se/post/ssdlc-architect-...,2025-09-23 21:09:42,True
2,Profinder-https://www.profinder.se/post/busine...,Profinder,https://www.profinder.se/post/business-analyst...,Business Analyst – Business Transformation,None,None,None,None,https://www.profinder.se/post/business-analyst...,2025-09-23 21:09:42,True
3,Profinder-https://www.profinder.se/post/fronte...,Profinder,https://www.profinder.se/post/frontendutveckla...,Frontendutvecklare med appkompetens,None,None,None,None,https://www.profinder.se/post/frontendutveckla...,2025-09-23 21:09:42,True
4,Profinder-https://www.profinder.se/post/backen...,Profinder,https://www.profinder.se/post/backendutvecklar...,Backendutvecklare API och Java,None,None,None,None,https://www.profinder.se/post/backendutvecklar...,2025-09-23 21:09:42,True
5,Profinder-https://www.profinder.se/post/c-utve...,Profinder,https://www.profinder.se/post/c-utvekcklare-id...,C++ utvekcklare,None,None,None,None,https://www.profinder.se/post/c-utvekcklare-id...,2025-09-23 21:09:42,True
6,Profinder-https://www.profinder.se/post/produk...,Profinder,https://www.profinder.se/post/produktionsingen...,Produktionsingenjör – Uppsala,None,None,None,None,https://www.profinder.se/post/produktionsingen...,2025-09-23 21:09:42,True
7,Profinder-https://www.profinder.se/post/networ...,Profinder,https://www.profinder.se/post/network-security...,Network Security Engineer,None,None,None,None,https://www.profinder.se/post/network-security...,2025-09-23 21:09:42,True
8,Profinder-https://www.profinder.se/post/networ...,Profinder,https://www.profinder.se/post/networking-cloud...,Networking & Cloud DevOps Engineer (SDAP Conne...,None,None,None,None,https://www.profinder.se/post/networking-cloud...,2025-09-23 21:09:42,True
9,Profinder-https://www.profinder.se/post/embedd...,Profinder,https://www.profinder.se/post/embedded-android...,Embedded Android,None,None,None,None,https://www.profinder.se/post/embedded-android...,2025-09-23 21:09:42,True


In [77]:
from playwright.sync_api import sync_playwright
from bs4 import BeautifulSoup

import re
import requests
from bs4 import BeautifulSoup
import re
import requests


class ProfinderScraper:

    def clean_title(self, title: str):
        """Remove 'ID:xxx' from title."""
        return re.sub(r"\s*ID:\d+", "", title).strip()

    def extract_id(self, title: str):
        """Extract job ID if present."""
        match = re.search(r"ID:(\d+)", title)
        return match.group(1) if match else None

    def extract_location(self, title: str):
        """Try to extract location if mentioned with 'i <Ort>'."""
        match = re.search(r" i ([A-ZÅÄÖa-zåäö\s-]+)", title)
        return match.group(1).strip() if match else None


    def scrape_all_jobs(self, max_pages: int = 5):
        all_jobs = []
        for page in range(1, max_pages + 1):
            data = self.get_page(page)
            posts = data.get("postFeedPage", {}).get("posts", {}).get("posts", [])
            if not posts:
                break
            for post in posts:
                title = post.get("title", "")
                job = {
                    "id": self.extract_id(title),
                    "title": self.clean_title(title),
                    "location": self.extract_location(title),
                    "excerpt": post.get("excerpt"),
                    "url": post.get("link"),
                    "published": post.get("firstPublishedDate"),
                }
                all_jobs.append(job)
            print(f"{self.__class__.site} > Scraped {len(posts)} jobs from page {page}")
        return all_jobs





## Load bronze table 

In [17]:
import pandas as pd
from src.data_loader import load_local_data, unload_local_data
bronze_data = load_local_data()
df = bronze_data.copy()
print(len(df['work_location'].unique()))
df['work_location'].unique()


123


array(['Göteborg', 'Jönköping', 'Kalmar län', 'Södermanlands län',
       'Stockholms län', 'Skåne län', 'Västra Götalands län',
       'Västernorrlands län Västerbottens län',
       'Västerbottens län Västernorrlands län', 'Östergötlands län',
       'Blekinge län', 'Västernorrlands län', nan, 'Västmanlands län',
       'Västmanlands län Stockholms län Uppsala län',
       'Dalarnas län Västra Götalands län Västmanlands län',
       'Norrbottens län', 'Örebro län', 'Stockholms län Kalmar län',
       'Stockholms län Uppsala län', 'Kalmar län Stockholms län',
       'Stockholms län Norrbottens län Västernorrlands län Västra Götalands län',
       'Västmanlands län Västernorrlands län Stockholms län',
       'Östergötlands län Gävleborgs län Dalarnas län',
       'Stockholms län Södermanlands län', 'Stockholm', 'Malmö/Lund',
       'Södertälje', 'Linköping', 'Boden', 'Karlskoga', 'Karlskrona',
       'Växjö', 'Västerås', 'Ludvika', 'Norrköping', 'Finspång',
       'Uppsala', 'Örebro', 

In [53]:
%pip install unidecode
%pip install rapidfuzz
%pip install geonamescache


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 51.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import re
import geonamescache
from unidecode import unidecode
from rapidfuzz import process
"""
Required packages to run in notebook on local machine:
%pip install unidecode
%pip install rapidfuzz
%pip install geonamescache
"""



['alingsås', 'björlanda', 'boden', 'boo', 'borlänge', 'borås', 'bromma', 'enköping', 'eskilstuna', 'eslöv', 'falkenberg', 'falköping', 'falun', 'gamla uppsala', 'gustavsberg', 'gävle', 'göteborg', 'halmstad', 'haninge', 'helsingborg', 'huddinge', 'hudiksvall', 'huskvarna', 'härnösand', 'hässleholm', 'jakobsberg', 'jönköping', 'kalmar', 'karlshamn', 'karlskoga', 'karlskrona', 'karlstad', 'katrineholm', 'kinna', 'kiruna', 'kristianstad', 'kristinehamn', 'kumla', 'kungsbacka', 'kungsholmen', 'kungälv', 'köping', 'landskrona', 'lerum', 'lidingö', 'lidköping', 'linköping', 'ljungby', 'ludvika', 'luleå', 'lund', 'majorna', 'malmö', 'mariestad', 'motala', 'märsta', 'mölndal', 'mölnlycke', 'nacka', 'norrköping', 'norrtälje', 'nyköping', 'nässjö', 'oskarshamn', 'partille', 'piteå', 'råsunda', 'sandviken', 'skellefteå', 'skövde', 'sollentuna', 'solna', 'staffanstorp', 'stockholm', 'sundbyberg', 'sundsvall', 'södermalm', 'södertälje', 'tensta', 'trelleborg', 'trollhättan', 'tullinge', 'tumba', 't